# Notebook de versement de données

Ce notebook permet de verser des données Excel dans une base PostgreSQL.

## 1. Importation des packages

In [ ]:
import sys
sys.path.append('..')

from script.body.base_donnees import ConnectionBaseDeDonnees
from script.body.livre import Livre
from script.body.traitement import Traitement
from script.body.versement import Versement
from script.branch.gestion_dossier import GestionDossier
from script.leaf.validator import DataValidator
from script.leaf.catalogue import Catalogue
from script.branch.gomme import Gomme

## 2. Configuration de la connexion

In [ ]:
# Connexion à la base de données
db = ConnectionBaseDeDonnees()
conn, schema = db.connexion_observatoire()

## 3. Configuration du versement

In [ ]:
# Paramètres du versement
fichier_excel = "hola.xlsx"
annee = 2025
theme = "theme"
base = "base"
source = "source"

# Validation du fichier
if not DataValidator.validate_excel_file(fichier_excel):
    raise ValueError("Fichier Excel invalide.")
else:
    print("Fichier Excel valide.")

# Enregistrement des paramètres du versement
livre = Livre(
    conn, 
    schema=schema, 
    theme=theme,
    base=base, 
    source=source, 
    annee=annee
)
livre.echelle = livre.choix_echelle()


## Premiere modification : gere seulement l'emplacement 


In [ ]:

# Paramètres du versement
fichier_excel = "uto.x"
annee = 2025
theme = "theme"
base = "base"
source = "source"

try:
    # Validation du fichier
    if not DataValidator.validate_excel_file(fichier_excel):
        raise ValueError("⚠️ Le fichier Excel fourni est invalide. Vérifiez son emplacement, son format et son contenu.")

    print("✅ Fichier Excel valide.")

    # Enregistrement des paramètres du versement
    livre = Livre(
        conn, 
        schema=schema, 
        theme=theme,
        base=base, 
        source=source, 
        annee=annee
    )
    livre.echelle = livre.choix_echelle()

except ValueError as e:
    print(f"🚨 Erreur : {e}")  # Affichage propre du message d'erreur 
                               # Pour éviter l’affichage des erreurs dans la sortie de la cellule


## Deuxieme test : Gere les trois erreur(Vérification de l'existence du fichier,Validation du format du fichier, Contrôle de l'intégrité du contenu)

In [ ]:
import os

def valider_fichier_excel(fichier):
    erreurs = []

    # Vérification de l'existence du fichier
    if not os.path.isfile(fichier):
        erreurs.append(f"Le fichier '{fichier}' est introuvable. Vérifiez le chemin spécifié.")

    # Validation du format du fichier
    if not fichier.endswith('.xlsx'):
        erreurs.append("Le fichier n'est pas au format '.xlsx'. Veuillez fournir un fichier Excel valide.")

    # Validation du contenu du fichier
    if not erreurs:  # Procéder à la validation du contenu uniquement si le fichier existe et a la bonne extension
        if not DataValidator.validate_excel_file(fichier):
            erreurs.append("Le contenu du fichier Excel est invalide. Vérifiez les données et la structure du fichier.")

    return erreurs

# Paramètres du versement
fichier_excel = "uto.xlsx"
annee = 2025
theme = "theme"
base = "base"
source = "source"

try:
    # Validation du fichier
    erreurs = valider_fichier_excel(fichier_excel)
    if erreurs:
        raise ValueError(erreurs)

    print("✅ Fichier Excel valide.")

    # Enregistrement des paramètres du versement
    livre = Livre(
        conn,
        schema=schema,
        theme=theme,
        base=base,
        source=source,
        annee=annee
    )
    livre.echelle = livre.choix_echelle()

except ValueError as e:
    print("🚨 Erreur(s) détectée(s) :")
    for erreur in e.args[0]:
        print(f" - {erreur}")
except Exception as e:
    print(f"🚨 Une erreur inattendue s'est produite : {e}")


## Catalogue / Facultatif

In [ ]:
catalogue = Catalogue(livre)
catalogue.liste_table(afficher=True)

In [ ]:
catalogue.catalogue()

## 4. Traitement des données

In [ ]:
# Traitement des données
traiteur = Traitement(conn, fichier_excel, livre)
traiteur.traitement()

## 5. Sauvegarde en base de données

In [ ]:
# Sauvegarde des données
versement = Versement(conn, livre)
versement.versement()

## 6. Suppression des fichiers temporaires

In [ ]:
gestionnaire_de_dossier = GestionDossier(livre)
gestionnaire_de_dossier.delete_file()

## 7. Suppression de table / versement

In [ ]:
gomme = Gomme(conn, livre)
gomme.suppression_table(table_cible='', id_versement_cible=None, garder_table=False)